[Home](../../index.ipynb) / [Web-Server](../index.ipynb) / Simple car: Test
***

# Simple car: Test

Nach dem Start der Programme das WLAN des Controllers auswählen und die auf der Console angezeigte IP-Adresse ansurfen (meist 192.168.4.1).

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 

In [ ]:
import machine

class Motor :
    def __init__( self, 
            iPinA, 
            iPinB,
            iFrequency     = 1000,
            iDutyMinLeft   =    0,  # Duty for speed = 0:              motor turns left
            iDutyMaxLeft   = 1023,  # Duty for speed = fSpeedMaxLeft:  motor turns left
            iDutyMinRight  =    0,  # Duty for speed = 0:              motor turns right
            iDutyMaxRight  = 1023,  # Duty for speed = fSpeedMaxRight: motor turns right
            fSpeedMaxLeft  = -1.0,  # Minimal value for setSpeed() --> iDutyMaxLeft
            fSpeedMaxRight =  1.0   # Maximal value for setSpeed() --> iDutyMaxRight
        ) :
        self._iFrequency    = iFrequency
        self._fScaleLeft    = (iDutyMaxLeft  - iDutyMinLeft )/fSpeedMaxLeft
        self._fScaleRight   = (iDutyMaxRight - iDutyMinRight)/fSpeedMaxRight
        self._iDutyMinLeft  = iDutyMinLeft
        self._iDutyMaxLeft  = iDutyMaxLeft
        self._iDutyMinRight = iDutyMinRight
        self._iDutyMaxRight = iDutyMaxRight
        
        self._fCurSpeed     = 0
        
        self._pwmA = machine.PWM( machine.Pin(iPinA, machine.Pin.OUT), freq=self._iFrequency, duty=0)
        self._pwmB = machine.PWM( machine.Pin(iPinB, machine.Pin.OUT), freq=self._iFrequency, duty=0)
        
        self._pwmDir   = self._pwmA
        self._pwmSpeed = self._pwmB
        
    def speedToDuty( self, fSpeed ) :
        if fSpeed == 0 :
            return 0
        elif fSpeed < 0 :
            return min( self._iDutyMaxLeft, max( self._iDutyMinLeft, int(self._iDutyMinLeft  + fSpeed*self._fScaleLeft +0.5) ) )
        else :
            return min( self._iDutyMaxRight, max( self._iDutyMinRight, int(self._iDutyMinRight + fSpeed*self._fScaleRight+0.5) ) )
            
    
    def setLeft( self ) :
        self._pwmDir   = self._pwmA
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmB
      
    def setRight( self ) :
        self._pwmDir   = self._pwmB
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmA
      
    def setSpeed( self, fSpeed ) :
        if fSpeed != self._fCurSpeed :
            iDuty = self.speedToDuty( fSpeed )
            
            if fSpeed > 0 and self._fCurSpeed <= 0 :
                self.setLeft()

            elif fSpeed < 0 and self._fCurSpeed >= 0 :
                self.setRight()
            
            self._pwmSpeed.duty( iDuty )
        
        self._fCurSpeed = fSpeed
        
    def deinit( self ):
        self.setSpeed( 0 )

        self._pwmSpeed.deinit()
        self._pwmDir.deinit()

In [ ]:
MAX_REQUEST_LENGTH = 1024

#================================================

import network
import socket
import info
import ubinascii
import re
import gc


#================================================

class ContentType :
    PLAIN      = "plain"
    HTML       = "html"
    CSS        = "css"
    JAVASCRIPT = "javascript"


#================================================

class RequestHandler :
    def __init__( self, strResponseType = "200 OK", strContentType = ContentType.HTML ) :
        self._strResponseType = strResponseType
        self._strContentType  = strContentType
        
    def dispatch( self, socketClient, strPath = None, match = None ) :
        pass
    
    def sendHeader( self, socketClient ) :
        socketClient.send( "HTTP/1.1 " )
        socketClient.send( self._strResponseType   )
        socketClient.send( "\nContent-type: text/" )
        socketClient.send( self._strContentType    )
        socketClient.send( "; charset=UTF-8\n"     )
        socketClient.send( "Connection: close\n\n" )

    def __str__( self ) :
        return "Name: " + self.__class__.__name__ + " Response type: " + self._strResponseType + " Content type: " + self._strContentType
    
#================================================
    

class StringRequestHandler( RequestHandler ) :
    def __init__( self, strData, strResponseType = "200 OK", strContentType = ContentType.HTML ) :
        super().__init__( strResponseType, strContentType )
        self._strData = strData
        
    def dispatch( self, socketClient, strPath = None, match = None ) :
        self.sendHeader( socketClient )
        socketClient.sendall( self._strData )

        
class FileRequestHandler( RequestHandler ) :
    def __init__( self, strFileName, strResponseType = "200 OK", strContentType = ContentType.HTML ) :
        super().__init__( strResponseType, strContentType )
        self._strFileName    = strFileName
        
    def dispatch( self, socketClient, strPath = None, match = None ) :
        self.sendHeader( socketClient )
        
        with open(self._strFileName, 'r') as f:
            while True:
                chunk = f.read(1024) # Read the next 1KB chunk
                if not chunk:
                    break
                socketClient.send(chunk) # Send the next 1KB chunk

            #socketClient.sendall( f.read() )

class Error404RequestHandler( FileRequestHandler ) :
    def __init__( self, strFileName, strContentType = ContentType.HTML ) :
        super().__init__( strFileName, "404 Not Found", strContentType )
        

#================================================
  
class SimpleAccessPointHttpServer:
    def __init__( self ) :
        self.ipAddress  = "?.?.?.?"
        self.subnetMask = self.ipAddress
        self.gateway    = self.ipAddress
        self.dnsServer  = self.ipAddress
        self.macAddress = "?:?:?:?:?:?"
        self.objSocket  = None
        
        self.aPathRequestHandler    = []
        self.error404RequestHandler = Error404RequestHandler( "/resources/error404.html" )
        
        
    def start( self, ssid = "ESP32 http server", password = "123123123123", maxClients = 5 ):
        self.wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface

        if info.TYPE == "Esp8266 Croduino Nova" :
            self.wlanAccessPoint.config( essid = ssid, password = password ) # Esp8266
        else :
            self.wlanAccessPoint.config( essid = ssid, password = password, max_clients = maxClients ) # ESP32

        self.wlanAccessPoint.active( True )                  # activate the interface

        print( self.wlanAccessPoint.ifconfig()[0] )

        self.objSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

        try:
            self.objSocket.bind( ( '', 80 ) )
            self.objSocket.listen( maxClients )
            
            self.ipAddress, self.subnetMask, self.gateway, self.dnsServer = self.wlanAccessPoint.ifconfig() # [IP address, subnet mask, gateway, DNS server]
            self.macAddress = ubinascii.hexlify( network.WLAN().config('mac'),':' ).decode()
            
        except Exception as e:
            #print( "Exception of type {0} occurred. Arguments:\n{1!r}".format(type(e).__name__, e.args))
            print( e )

        gc.collect()

        
    def stop( self ):
        try : self.objSocket.close()
        except: pass
    
        try : self.wlanAccessPoint.active( False ) 
        except: pass
    
        gc.collect()

    
    def dispatch( self, socketClient ) :
        bDispatched = False
        
        strPath = self.getRequestPath( str( socketClient.recv( MAX_REQUEST_LENGTH ) ) )

        for pathRequestHandler in self.aPathRequestHandler :
            match = pathRequestHandler[0].match( strPath ) # re.match( r'(/path/)(.*)', line)
            
            #print( "dispatch: " + strPath + " - " + str( match ) )
            
            if match :
                #print( "match: " +  strPath )
                pathRequestHandler[1].dispatch( socketClient, strPath, match )
                bDispatched = True
                break
            # else :
                # print( "not match: " + _strPath + " - "  + str( pathRequestHandler[1] ) )
                
        if bDispatched == False :
            self.error404RequestHandler.dispatch( _connection )
            

    def accept( self ):
        socketClient = None
        
        try :
            # socket.accept(): The return value is a pair (socketClient, addressClient)
            # where socketClient is a new socket object we can send data to, or read data from
    
            socketClient = self.objSocket.accept()[ 0 ]
        
            self.dispatch( socketClient )
            
        except Exception as _e:
            print( _e )
            #print( "Exception of type {0} occurred. Arguments:\n{1!r}".format(type(_e).__name__, _e.args))
        
        finally :
            #print(socketClient)
            try : socketClient.close()
            except: pass

        
    def getRequestPath( self, strRequest ) :
        # url = ure.search("(?:GET|POST) /(.*?)(?:\\?.*?)? HTTP", request).group(1).decode("utf-8").rstrip("/")
        iRequestPathStart = strRequest.find( "GET " )

        if iRequestPathStart >= 0 :
            iRequestPathStart += 4
            
            iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

            if iRequestPathEnd >= 0 :
                return strRequest[ iRequestPathStart : iRequestPathEnd ]
            
        return "/"
    
 
    def addRequestHandler( self, strPath, requestHandler ) :
        self.aPathRequestHandler.append( ( re.compile( strPath ), requestHandler ) )

    def __str__( self ) :
        return "Name: " + self.__class__.__name__ + ", IP address: " + self.ipAddress + ", Subnet mask: " + self.subnetMask + ", Gateway: " + self.gateway + ", DNS server: " + self.dnsServer + ", MAC address: " + self.macAddress
        

In [ ]:
# import httpServer
# import motor

class SpeedRequestHandler( RequestHandler ) :
    def __init__( self ) :
        super().__init__( strContentType = ContentType.PLAIN )
        self._fCurV    = 0
        self._fCurGear = 0
        self._motorL = Motor( 12, 13,
            iFrequency     = 100,
            iDutyMinLeft   = 330,
            iDutyMaxLeft   = 1023,
            iDutyMinRight  = 330,
            iDutyMaxRight  = 1023,
            fSpeedMaxLeft  = -1.0,
            fSpeedMaxRight =  1.0 )

        self._motorR = Motor( 18, 22,
            iFrequency     = 100,
            iDutyMinLeft   = 330,
            iDutyMaxLeft   = 1023,
            iDutyMinRight  = 330,
            iDutyMaxRight  = 1023,
            fSpeedMaxLeft  = -1.0,
            fSpeedMaxRight =  1.0 )
        
        
    def dispatch( self, socketClient, strPath = None, match = None ) :
        # Here You have velocity in[-1;1] and gear in [-1;1]
        
        fV    = float(match.group( 1 ))
        fGear = float(match.group( 2 ))
        
        if fV != self._fCurV or fGear != self._fCurGear :
            self._motorL.setSpeed( fV + fGear )
            self._motorR.setSpeed( fV - fGear )
            
            self._fCurV    = fV
            self._fCurGear = fGear
            
            #print( "Set: Velocity = {}, Gear = {}".format( fV, fGear ) )
            
            return True
        
        

# httpServer = httpServer.SimpleAccessPointHttpServer()
httpServer = SimpleAccessPointHttpServer()


httpServer.addRequestHandler( "/joystick/(.*)_(.*)", SpeedRequestHandler() )
httpServer.addRequestHandler( "/default.css", FileRequestHandler( "/resources/default.css", strContentType = ContentType.CSS        ) )
httpServer.addRequestHandler( "/default.js",  FileRequestHandler( "/resources/default.js",  strContentType = ContentType.JAVASCRIPT ) )
httpServer.addRequestHandler( "/",            FileRequestHandler( "/resources/index.html" ) )


httpServer.start( ssid = "ESP32 Simple car", password = "123123123123" )

print( httpServer )


#######################################
# Loop

try :
    while True:
        print( "Waiting for client...", end="" )
        httpServer.accept()
        
except KeyboardInterrupt:
    httpServer.stop()
    print( "Server stopped." )

In [ ]:
%serialconnect
%sendtofile  --source "resources"
# %sendtofile --mkdir --source "resources/index.html"
#%sendtofile    resources/error404.html
#%sendtofile    resources/default.js
#%sendtofile    resources/default.css

In [ ]:
%ls resources


In [ ]:
%serialconnect

print(open("/resources/index.html").read())
print( "\n####\n" )
print(open("/resources/default.css").read())
print( "\n####\n" )
print(open("/resources/default.js").read())
print( "\n####\n" )
print(open("/resources/error404.html").read())
print( "\n####\n" )


In [13]:
%ls "resources"


Listing directory 'resources'.
     1124    resources/default.css
     4528    resources/default.js
      533    resources/error404.html
      762    resources/index.html


In [12]:
%serialconnect
%sendtofile  --source "resources"


Sent 14 lines (533 bytes) to resources/.ipynb_checkpoints/error404-checkpoint.html.


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'O' isn't defined


In [14]:
print(open("/resources/default.js").read())

document.addEventListener(
    'DOMContentLoaded',
    function() { new JoystickApp(); },
    false
);


class JoystickApp
{
    constructor()
    {
        let objThis = this;

        this.isActive = false;
        this.posX   = 0;
        this.posY   = 0;
        this.startX = 0;
        this.startY = 0;

        this.htmlMainContainer  = document;
        this.htmlStickContainer = document.getElementById( "idStickContainer" );
        this.htmlStick          = document.getElementById( "idStick");

        this.radiusMax = this.htmlStickContainer.clientWidth/2 - 40;

        this.htmlMainContainer.addEventListener("touchstart", this.dragStart.bind(this), false);
        this.htmlMainContainer.addEventListener("touchend",   this.dragEnd.bind(this), false);
        this.htmlMainContainer.addEventListener("touchmove",  this.drag.bind(this), false);

        this.htmlMainContainer.addEventListener("mousedown",  this.dragStart.bind(this), false);
        this.htmlMainContainer.addEventLi